In [1]:
import datetime
from pyspark.sql import DataFrame, Row
from pyspark.sql.types import *
from pyspark.sql.functions import unix_timestamp, from_unixtime, col
from pyspark.sql import functions as F
import sys
from hops import hdfs as hdfs
print(hdfs.project_path())

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
11,application_1613204927129_0015,pyspark,idle,Link,Link


SparkSession available as 'spark'.
hdfs://rpc.namenode.service.consul:8020/Projects/pittre/

In [7]:
fg1_schema = StructType([
  StructField("id", IntegerType(), True),
  StructField("ts", IntegerType(), True),
  StructField("f1", StringType(), True)    
])

fg1=spark.read.csv("hdfs:///Projects/" + hdfs.project_name() + "/Resources/5-10-2-out.csv", header=True, schema=fg1_schema)
fg1=fg1.sort(col("id"),col("ts"))
fg1.show()

+---+----+-----+
| id|  ts|   f1|
+---+----+-----+
|  1|1002| f2-1|
|  1|1004| f2-2|
|  1|1006| f2-3|
|  1|1008| f2-4|
|  1|1010| f2-5|
|  2|1002| f2-6|
|  2|1004| f2-7|
|  2|1006| f2-8|
|  2|1008| f2-9|
|  2|1010|f2-10|
|  3|1002|f2-11|
|  3|1004|f2-12|
|  3|1006|f2-13|
|  3|1008|f2-14|
|  3|1010|f2-15|
|  4|1002|f2-16|
|  4|1004|f2-17|
|  4|1006|f2-18|
|  4|1008|f2-19|
|  4|1010|f2-20|
+---+----+-----+
only showing top 20 rows

In [8]:
fg2_schema = StructType([
  StructField("id", IntegerType(), True),
  StructField("ts", IntegerType(), True),
  StructField("f2", StringType(), True)    
])

fg2=spark.read.csv("hdfs:///Projects/" + hdfs.project_name() + "/Resources/5-10-3-out.csv", header=True, schema=fg2_schema)
fg2.show()

+---+----+-----+
| id|  ts|   f2|
+---+----+-----+
|  1|1003| f3-1|
|  1|1006| f3-2|
|  1|1009| f3-3|
|  2|1003| f3-4|
|  2|1006| f3-5|
|  2|1009| f3-6|
|  3|1003| f3-7|
|  3|1006| f3-8|
|  3|1009| f3-9|
|  4|1003|f3-10|
|  4|1006|f3-11|
|  4|1009|f3-12|
|  5|1003|f3-13|
|  5|1006|f3-14|
|  5|1009|f3-15|
+---+----+-----+

In [9]:
joined=fg1.join(fg2, (fg1.id == fg2.id) & (fg1.ts == fg2.ts), how="full_outer").orderBy(fg1.id, fg1.ts)
joined.show()

+----+----+-----+----+----+-----+
|  id|  ts|   f1|  id|  ts|   f2|
+----+----+-----+----+----+-----+
|null|null| null|   5|1003|f3-13|
|null|null| null|   5|1009|f3-15|
|null|null| null|   1|1009| f3-3|
|null|null| null|   4|1003|f3-10|
|null|null| null|   2|1009| f3-6|
|null|null| null|   3|1003| f3-7|
|null|null| null|   3|1009| f3-9|
|null|null| null|   1|1003| f3-1|
|null|null| null|   2|1003| f3-4|
|null|null| null|   4|1009|f3-12|
|   1|1002| f2-1|null|null| null|
|   1|1004| f2-2|null|null| null|
|   1|1006| f2-3|   1|1006| f3-2|
|   1|1008| f2-4|null|null| null|
|   1|1010| f2-5|null|null| null|
|   2|1002| f2-6|null|null| null|
|   2|1004| f2-7|null|null| null|
|   2|1006| f2-8|   2|1006| f3-5|
|   2|1008| f2-9|null|null| null|
|   2|1010|f2-10|null|null| null|
+----+----+-----+----+----+-----+
only showing top 20 rows

In [10]:
joined=fg1.join(fg2, (fg1.id == fg2.id) & (fg1.ts == fg2.ts), how="left_outer").orderBy(fg1.id, fg1.ts)
joined.show()

+---+----+-----+----+----+-----+
| id|  ts|   f1|  id|  ts|   f2|
+---+----+-----+----+----+-----+
|  1|1002| f2-1|null|null| null|
|  1|1004| f2-2|null|null| null|
|  1|1006| f2-3|   1|1006| f3-2|
|  1|1008| f2-4|null|null| null|
|  1|1010| f2-5|null|null| null|
|  2|1002| f2-6|null|null| null|
|  2|1004| f2-7|null|null| null|
|  2|1006| f2-8|   2|1006| f3-5|
|  2|1008| f2-9|null|null| null|
|  2|1010|f2-10|null|null| null|
|  3|1002|f2-11|null|null| null|
|  3|1004|f2-12|null|null| null|
|  3|1006|f2-13|   3|1006| f3-8|
|  3|1008|f2-14|null|null| null|
|  3|1010|f2-15|null|null| null|
|  4|1002|f2-16|null|null| null|
|  4|1004|f2-17|null|null| null|
|  4|1006|f2-18|   4|1006|f3-11|
|  4|1008|f2-19|null|null| null|
|  4|1010|f2-20|null|null| null|
+---+----+-----+----+----+-----+
only showing top 20 rows

In [14]:
fg1a=fg1.withColumnRenamed("id", "id_1").withColumnRenamed("ts", "ts_1")
joined=fg1a.join(fg2, (fg1a.id_1 == fg2.id) & (fg1a.ts_1 == fg2.ts), how="full_outer").orderBy(fg1a.id_1, fg1a.ts_1)
joined.show()

+----+----+-----+----+----+-----+
|id_1|ts_1|   f1|  id|  ts|   f2|
+----+----+-----+----+----+-----+
|null|null| null|   5|1003|f3-13|
|null|null| null|   5|1009|f3-15|
|null|null| null|   1|1009| f3-3|
|null|null| null|   4|1003|f3-10|
|null|null| null|   2|1009| f3-6|
|null|null| null|   3|1003| f3-7|
|null|null| null|   3|1009| f3-9|
|null|null| null|   1|1003| f3-1|
|null|null| null|   2|1003| f3-4|
|null|null| null|   4|1009|f3-12|
|   1|1002| f2-1|null|null| null|
|   1|1004| f2-2|null|null| null|
|   1|1006| f2-3|   1|1006| f3-2|
|   1|1008| f2-4|null|null| null|
|   1|1010| f2-5|null|null| null|
|   2|1002| f2-6|null|null| null|
|   2|1004| f2-7|null|null| null|
|   2|1006| f2-8|   2|1006| f3-5|
|   2|1008| f2-9|null|null| null|
|   2|1010|f2-10|null|null| null|
+----+----+-----+----+----+-----+
only showing top 20 rows

In [24]:
from pyspark.sql import functions as F
joined2 = joined.withColumn('Id_New',F.when(joined.id_1.isNull(),joined.id).otherwise(joined.id_1)).drop(joined.id_1).drop(joined.id).na.fill(0).na.fill("")
joined2.orderBy(joined2.Id_New).show()

+----+-----+----+----+------+
|ts_1|   f1|  ts|  f2|Id_New|
+----+-----+----+----+------+
|1004| f2-2|   0|    |     1|
|1008| f2-4|   0|    |     1|
|1010| f2-5|   0|    |     1|
|   0|     |1003|f3-1|     1|
|   0|     |1009|f3-3|     1|
|1006| f2-3|1006|f3-2|     1|
|1002| f2-1|   0|    |     1|
|1010|f2-10|   0|    |     2|
|1004| f2-7|   0|    |     2|
|1006| f2-8|1006|f3-5|     2|
|1008| f2-9|   0|    |     2|
|   0|     |1009|f3-6|     2|
|1002| f2-6|   0|    |     2|
|   0|     |1003|f3-4|     2|
|1002|f2-11|   0|    |     3|
|1006|f2-13|1006|f3-8|     3|
|1010|f2-15|   0|    |     3|
|1004|f2-12|   0|    |     3|
|   0|     |1009|f3-9|     3|
|1008|f2-14|   0|    |     3|
+----+-----+----+----+------+
only showing top 20 rows

In [25]:
from pyspark.sql import functions as F

grouped_rdd = joined2.groupBy(joined2.Id_New).agg(F.collect_list(joined2.ts_1), F.collect_list(joined2.f1), F.collect_list(joined2.f2), F.collect_list(joined2.ts))
grouped_rdd.collect()

[Row(Id_New=1, collect_list(ts_1)=[0, 0, 1002, 1004, 1006, 1008, 1010], collect_list(f1)=['', '', 'f2-1', 'f2-2', 'f2-3', 'f2-4', 'f2-5'], collect_list(f2)=['f3-1', 'f3-3', '', '', 'f3-2', '', ''], collect_list(ts)=[1003, 1009, 0, 0, 1006, 0, 0]), Row(Id_New=3, collect_list(ts_1)=[0, 0, 1002, 1004, 1006, 1008, 1010], collect_list(f1)=['', '', 'f2-11', 'f2-12', 'f2-13', 'f2-14', 'f2-15'], collect_list(f2)=['f3-7', 'f3-9', '', '', 'f3-8', '', ''], collect_list(ts)=[1003, 1009, 0, 0, 1006, 0, 0]), Row(Id_New=5, collect_list(ts_1)=[0, 0, 1002, 1004, 1006, 1008, 1010], collect_list(f1)=['', '', 'f2-21', 'f2-22', 'f2-23', 'f2-24', 'f2-25'], collect_list(f2)=['f3-15', 'f3-13', '', '', 'f3-14', '', ''], collect_list(ts)=[1009, 1003, 0, 0, 1006, 0, 0]), Row(Id_New=4, collect_list(ts_1)=[0, 0, 1002, 1004, 1006, 1008, 1010], collect_list(f1)=['', '', 'f2-16', 'f2-17', 'f2-18', 'f2-19', 'f2-20'], collect_list(f2)=['f3-12', 'f3-10', '', '', 'f3-11', '', ''], collect_list(ts)=[1009, 1003, 0, 0, 1006

In [ ]:
def align(ts_1, f1, f2, ts_2):
    latest_f2=0
    for i in range(len(ts_1)):
        if f2[i] != null:
            latest_f2=f2
        

In [ ]:
maprdd = grouped_rdd.map(lambda x:(x[0],{y[1] for y in x[1]}))


In [10]:
maprdd = fg2.rdd.groupBy(lambda x:x[0]).map(lambda x:(x[0],{y[1] for y in x[1]}))
result_dict = dict(maprdd.collect()) 
result_dict

{1: {1611505152, 1611505169, 1611505154, 1611505155}, 2: {1611505154, 1611505162, 1611505159}, 3: {1611505163}, 4: {1611505165}}

In [11]:
map_broadcast = sc.broadcast(result_dict)

In [32]:
def take_closest(id, ts):
    return min(map_broadcast.value[id],key=lambda x:((ts-x) if ts >= x else sys.maxsize))
columns = ["id2","join_ts","ts2"]

my_rdd = fg1.rdd.map(lambda x: (x[0],x[1],take_closest(x[0],x[1])))
filtered_rdd = my_rdd.filter(lambda x: x[1] >= x[2])
join2=spark.createDataFrame(filtered_rdd,columns).sort(col("id2"),col("join_ts"))

# columns_to_drop = ['ts']
# join2 = join2.drop(*columns_to_drop)
join2.show()

+---+----------+----------+
|id2|   join_ts|       ts2|
+---+----------+----------+
|  1|1611505155|1611505155|
|  1|1611505156|1611505155|
|  2|1611505157|1611505154|
|  2|1611505161|1611505159|
+---+----------+----------+

In [36]:
joined = fg1.join(join2,(fg1.id==join2.id2) & (fg1.ts==join2.join_ts),how="inner")
drop_cols = ['id2', 'join_ts']
inter = joined.drop(*drop_cols)
inter.show(15)

+---+----------+---+----------+
| id|        ts| f1|       ts2|
+---+----------+---+----------+
|  1|1611505155|f11|1611505155|
|  1|1611505156|f12|1611505155|
|  2|1611505157|f21|1611505154|
|  2|1611505161|f22|1611505159|
+---+----------+---+----------+

In [44]:
inter = inter.alias('inter')
fg2 = fg2.alias('fg2')
final = inter.join(fg2,(inter.id==fg2.id) & (inter.ts2==fg2.ts),how="inner").select('inter.id', 'inter.ts', 'inter.f1', 'fg2.f2')
final.show(15)

+---+----------+---+----+
| id|        ts| f1|  f2|
+---+----------+---+----+
|  1|1611505155|f11|f213|
|  1|1611505156|f12|f213|
|  2|1611505157|f21|f220|
|  2|1611505161|f22|f221|
+---+----------+---+----+